### Change this stuff

In [1]:
# choose "year and month" or "year and quarter"
report_type = 'year and month'

# choose title
# if report_type == year_and_month, do YYYY - MM - three_leter_month_name - Monthly Performance Report
# if report_type == year_and_quarter, do YYYY - Q[1-4] - Quarter Performance Report
report_title = '2018 - 02 - Feb - Monthly Performance Report'

# do you want to run the script AND generate the PDF report? or just run the script?
# the script will run regardless of what you choose here
# choose "yes" or "no"
pdf_write = 'yes'

### Libraries

In [2]:
import os
tilde = os.path.expanduser('~')

import sys
sys.path.insert(0, tilde + '/Scripts/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

from part_cost_data import *
from scipy.interpolate import interp1d


== part_cost_data start ==

0 nulls
the mean_stripe_margin is 77.44%

the part types with no MSRP are/is: ['sku']

avg gross profit for:
parts, combos, and stripes
which have been purchased within the last year
whose sku_status equals "working"
whose msrp is greater than zero
which were bought by non resellers
is 55.08%

the parts with negative gross profit are: [2885, 3400, 3834]

your dfs are: all_cost (cost on the sku level) and all_cost_by_part (cost on the part level)

== part_cost_data end ==



### Script settings

In [64]:
html_title = 'The Pillars Ratio Analysis.html'

now = dt.datetime.now()
now_pretty = str(now.date())

csv_path = tilde + '/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis/CSVs/'
html_path = tilde + '/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis/HTML/'
style_sheet_path = tilde + '/Scripts/Fake Folder/CSS Stylesheets/Recurring Reports Stylesheet.css'
pdf_path = tilde + '/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis/Docs/Reports/'
image_path = tilde + '/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis/Images/'

### Resources

[Online Business Dictionary](http://www.businessdictionary.com/)

[PyEmoji](https://pypi.org/project/emoji/)

### Get QB Main data
Which is generated in the "Pillars Keynote and Summary" Python script

In [4]:
t = 'QB Main for Ratio Analysis'
qb_main = pd.read_excel(csv_path + t + '.xlsx')

### Get Balance Sheet data

In [5]:
all_files = glob.glob(csv_path + '*.csv')
frame = pd.DataFrame()
list_ = []

for file_ in all_files:
    if 'Balance Sheet' in file_:
        df = pd.read_csv(file_, low_memory = False)
        list_.append(df)
    
balance_super_main = pd.concat(list_, sort = False)

balance_super_main.columns = balance_super_main.columns.str.lower()
for col in balance_super_main.columns:
    try:
        balance_super_main[col] = balance_super_main[col].str.lower()
    except:
        pass
    
# turn to numbers
for x in ['amount','balance']:
    balance_super_main[x] = balance_super_main[x].str.replace(',','')
    balance_super_main[x] = pd.to_numeric(balance_super_main[x])    

### Make a copy

In [6]:
balance_main = balance_super_main.copy()

### Find and remove rows which contain "total"s
The totals from QuickBooks are the totals over the ENTIRE period for which we pulled data, the entire period often being several years. These QB totals are irrelevant to us because we want the freedom to calculate totals over whatever period we choose (one year, one quarter, 6 months, etc.)

In [7]:
# all columns before "date" are susceptible
n = balance_super_main.columns.get_loc('date')
cols = balance_super_main.columns.tolist()[:n]

# get the index of all "totals"
totals_ix = []
for c in cols:
    df = balance_main[[c]]
    try:
        for i in df[df[c].str.contains('^total', na = False)].index:
            totals_ix.append(i)
    except:
        pass
    
balance_main.drop(totals_ix, inplace = True)
balance_main.reset_index(drop = True, inplace = True)

### Structure

In [8]:
# the format of the CSV output of QB data is such that two columns of nulls
    # separate the transaction labels and the transaction data.
    # below, get this first column so we know where to start our row-labeling
col = balance_main.loc[:, balance_main.isnull().all()].columns[0]
n = balance_main.columns.get_loc(col)

# fill NANs row wise
for i in np.arange(n-1,0,-1):
    balance_main[balance_main.columns[i]].fillna(balance_main[balance_main.columns[i-1]], inplace = True)
    
# fill NANs column wise
for i in np.arange(0,n):
    balance_main[balance_main.columns[i]].fillna(method = 'ffill', inplace = True)

# drop any rows/cols of all NANs
for i in range(2):
    balance_main.dropna(how = 'all', inplace = True, axis = i)

# rename columns
cols = ['super category',
        'main category',
        'subcategory 01',
        'subcategory 02',
        'subcategory 03',
        'subcategory 04']

for i in range(len(cols)):
    balance_main.rename(columns = {balance_main.columns[i]: cols[i]}, inplace = True)
    balance_main[cols[i]].fillna('none', inplace = True)
    
# further correct for the way excel formats the deeper levels of the hieracrhy        
s1 = set(balance_main[[x.isdigit() for x in balance_main['subcategory 02'].str[:5]]].index)
s2 = set(balance_main[[x.isdigit() for x in balance_main['subcategory 03'].str[:5]]].index)

ix1 = list(s1.intersection(s2))

df1 = balance_main.loc[ix1]
df2 = df1[df1['subcategory 02'].str[:3] != df1['subcategory 03'].str[:3]]

ix2 = df2.index.tolist()

for i in [3,4]:
    balance_main.loc[ix2, 'subcategory 0' + str(i)] = 'none'    

### Create date parameters

In [9]:
balance_main['date'] = pd.to_datetime(balance_main['date'])

date_start = str(balance_main['date'].min().date())
date_end = str(balance_main['date'].max().date())

### Patch up some things
We pulled data from QuickBooks between two dates, call them date_start and date_end. If some entry was transacted before this date_start, the "amount" field will be null during this period, but the "balance" field will contain a value. The section below finds these transactions.

In [10]:
patch_main = balance_main[(balance_main['date'].isnull())
                        & (balance_main['balance'].isnull() == False)
                        & (balance_main['balance'] != 0)
                        & (balance_main['subcategory 04'] != 'none')]

ls1 = list(patch_main['subcategory 04'])

ls2 = []
for i in [3,4]:
    inter_ls = balance_main[balance_main['date'].isnull() == False]['subcategory 0' + str(i)].tolist()
    for j in inter_ls:
        ls2.append(j)
        
ls3 = list(set(ls1) - set(ls2))

# we know from experience that this entry does not count; let's remove it
x = '20000 · accounts payable'
ls3.remove(x)

ls3

['30109 · jetblue']

Give these entries the date_start

In [11]:
balance_main['date'] = np.where(balance_main['subcategory 03'].isin(ls3) | balance_main['subcategory 04'].isin(ls3),
                                date_start,
                                balance_main['date'].map(str))

### Create dates

In [12]:
balance_main['date'] = pd.to_datetime(balance_main['date'])
balance_main['year'] = balance_main['date'].dt.year
balance_main['year and month'] = balance_main['date'].map(str).str[:7]
balance_main['year and quarter'] = balance_main['date'].map(str).str[:4] + '-Q' + balance_main['date'].dt.quarter.map(str)

### Check totals
Don't do equity, for equity QB factors in net income, where  
(net income) = (total income) - COGS - (operating expenses)  
but in the pillars scripts we define COGS differently than QB does, so this total will not match

In [13]:
total_assets = balance_super_main[balance_super_main['unnamed: 0'] == 'total assets']['amount'].item()
structured_assets = balance_main[balance_main['super category'] == 'assets'][['amount']].sum().item()

if np.abs(total_assets - structured_assets) > 0.001:
    raise ValueError('your asset totals don\'t match')

total_liabilties = balance_super_main[balance_super_main['unnamed: 1'] == 'total liabilities']['amount'].item()
structured_liabilties = balance_main[balance_main['main category'] == 'liabilities'][['amount']].sum().item()

if np.abs(total_liabilties - structured_liabilties) > 0.001:
    raise ValueError('your asset totals don\'t match')

### View hierarchy

In [14]:
view = 'no'
if view == 'yes':
    display(balance_main.groupby(cols)[['amount']].sum().format_(['m0']))

# Get accounts receivable data

In [15]:
# non partials
osh1 = pd.read_sql(
'''
SELECT
DATE(date_added) AS date,
orders_id,
0 AS partial_id,
orders_billing_id AS billing_status_id
FROM orders_status_history
WHERE DATE(date_added) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
''', db)

col_fix(osh1)

# partials
osh2 = pd.read_sql(
'''
SELECT
DATE(psh_date_added) AS date,
psh_parent_id AS orders_id,
psh_partial_id AS partial_id,
psh_billing_id AS billing_status_id
FROM partial_status_history
WHERE DATE(psh_date_added) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
''', db)

col_fix(osh2)

# drop any partial OIDs that are in non partial data
osh1.drop(osh1[osh1['orders id'].isin(osh2['orders id'].tolist())].index, inplace = True)
osh1.reset_index(drop = True, inplace = True)

# billing statuses
bs = pd.read_sql(
'''
SELECT
bs_status_id AS billing_status_id,
LOWER(bs_status_name) AS billing_status_name
FROM billing_status
''', db)

col_fix(bs)

# concat
osh_main = pd.concat([osh1, osh2], sort = True)

# create some columns
osh_main['billing status name'] = osh_main['billing status id'].map(dict(zip(bs['billing status id'], bs['billing status name'])))
osh_main['umbrella orders id'] = osh_main['orders id'].map(str) + '-' + osh_main['partial id'].map(str)
osh_main['date'] = pd.to_datetime(osh_main['date'])
osh_main['year and month'] = osh_main['date'].map(str).str[:7]
osh_main['year and quarter'] = osh_main['date'].map(str).str[:4] + '-Q' + osh_main['date'].dt.quarter.map(str)

# drop zeros
osh_main.drop(osh_main[osh_main['orders id'] == 0].index, inplace = True)
osh_main.reset_index(drop = True, inplace = True)

### Check out billing status name counts

In [16]:
osh_main['billing status name'].value_counts()

not set                    4060418
charged                    1001581
authorized                  685316
not billed                   29930
none required                26810
billed                       15732
payment received             15653
refunded                      9324
in collections                 392
write off                        9
multiple billing                 6
paid - multiple billing          2
Name: billing status name, dtype: int64

### Note
The data suggests that the addition of the "billing status" field is much more recent than the "orders status" field.

For this section, we only want to note when an order was marked as "billed" and "payment recieved".

But there are some OIDs whose "orders status" and "billing status" fields have conflicting info as to when the OID was marked as "billed" or "received".

[OID 628717](https://volcano.adafruit.com/volcano/Order_Status.php?oid=628717) is a good example.

We want to use the "billing status" field, so we must exclude the OIDs which have conflicting data.

### Do this below...

In [17]:
# from our osh_main data, get only the entries where the billing status is "billed" or "received"
ls = ['billed',
      'payment received']

osh = osh_main[osh_main['billing status name'].isin(ls)].copy()

# create a list of these OIDs
oid_ls = list(set(osh['orders id']))

# get the max OID which has conflicting data
cut_off = pd.read_sql(
'''
SELECT
MAX(orders_id) AS max_oid
FROM orders_status_history
WHERE orders_id IN '''+ str(tuple(oid_ls)) +'''
AND orders_status_id IN (6,7) # billed, payment received
''', db)

col_fix(cut_off)

# this is the OID we will use to exclude the OIDs with conflicting data
demarcation_oid = cut_off['max oid'][0]

# get some counts to see how many you are excluding
a = len(osh[osh['orders id'] <= demarcation_oid])
b = len(osh[osh['orders id'] > demarcation_oid])

print('you will exclude {:,.0f} OIDs'.format(a))
print('you will keep {:,.0f} OIDs'.format(b))

osh.drop(osh[osh['orders id'] <= demarcation_oid].index, inplace = True)
osh.reset_index(drop = True, inplace = True)

you will exclude 526 OIDs
you will keep 30,859 OIDs


### Get order totals of each umbrella OID

In [18]:
min_oid = osh['orders id'].min()

In [19]:
# not partials
o1 = pd.read_sql(
'''
SELECT
orders_id,
0 AS partial_id,
value AS net_amount # net amount after all deductions like coupons, etc.
FROM orders_total
WHERE class = 'ot_total'
AND orders_id >= ''' + str(min_oid) + '''
''', db)

col_fix(o1)

# partials
o2 = pd.read_sql(
'''
SELECT
pt_parent_id AS orders_id,
pt_partial_id AS partial_id,
pt_value AS net_amount # this is the net amount after any deductions like coupons, etc
FROM partial_total
WHERE pt_class = 'pt_total'
AND pt_parent_id >= ''' + str(min_oid) + '''
''', db)

col_fix(o2)
   
# remove partials from non partials data
o1.drop(o1[o1['orders id'].isin(o2['orders id'].tolist())].index, inplace = True)
o1.reset_index(drop = True, inplace = True)

# concat into one
cred_totals = pd.concat([o1,o2], sort = True)

# create columns
cred_totals['umbrella orders id'] = cred_totals['orders id'].map(str) + '-' + cred_totals['partial id'].map(str)

# check dupes
d = cred_totals[cred_totals['umbrella orders id'].duplicated()]
print('{:,.0f} dupe umbrella orders id(s)'.format(len(d)))
cred_totals.drop_duplicates('umbrella orders id', inplace = True)

# map net amount
osh['net amount'] = osh['umbrella orders id'].map(dict(zip(cred_totals['umbrella orders id'], cred_totals['net amount'])))

# check for nulls
n = osh[osh.isnull().any(1)]
print('{:,.0f} null(s) after mapping'.format(len(n)))
osh.dropna(inplace = True)
osh.reset_index(drop = True, inplace = True)

# drop dupes
osh.sort_values(['umbrella orders id','billing status name','date'], inplace = True)
osh.drop_duplicates(['umbrella orders id','billing status name'], keep = 'first', inplace = True)
osh.reset_index(drop = True, inplace = True)

1 dupe umbrella orders id(s)
4 null(s) after mapping


# Get accounts payable data

In [20]:
# POs and payment statuses
po_history = pd.read_sql(
'''
SELECT
DATE(po_history_date_modified) AS date,
po_id,
LOWER(payment_status) AS payment_status
FROM po_history
WHERE DATE(po_history_date_modified) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND po_id > 0
''', db)

col_fix(po_history)

# drop dupes
po_history.sort_values(['date','po id','payment status'], inplace = True)
po_history.drop_duplicates(['po id','payment status'], keep = 'first', inplace = True)
po_history.reset_index(drop = True, inplace = True)

# get min PO
min_po = po_history['po id'].min()

# get PO cost
po_stuff = pd.read_sql(
'''
SELECT
po_id,
po_stuff_adjusted_cost * po_stuff_qty AS po_cost
FROM po_stuff
WHERE po_id >= '''+ str(min_po) +'''
''', db)

col_fix(po_stuff)

# map PO cost
po_history['po cost'] = po_history['po id'].map(dict(zip(po_stuff['po id'], po_stuff['po cost'])))

# check for nulls
n = po_history[po_history.isnull().any(1)]
print('{:,.0f} null(s) after mapping'.format(len(n)))
po_history.dropna(inplace = True)
po_history.reset_index(drop = True, inplace = True)

# get fees
po_fees = pd.read_sql(
'''
SELECT
po_id,
po_fees_amount,
po_fees_name
FROM po_fees
WHERE po_id >= '''+ str(min_po) +'''
''', db)

col_fix(po_fees)

# map fees
cols = po_fees.columns.tolist()
cols.remove('po id')
for c in cols:
    po_history[c] = po_history['po id'].map(dict(zip(po_fees['po id'], po_fees[c]))).fillna(0)
    
# note that this net_order_total includes any deductions like credits or returns, etc.    
po_history['net order total'] = po_history[['po cost','po fees amount']].sum(1)

# create payable main df
# note that we only count POs that are FULLY PAID
# to my knowledge there is now way to see how much was partially paid
payable_main = po_history[po_history['payment status'] == 'invoice paid'].copy()

# creat columns
payable_main['date'] = pd.to_datetime(payable_main['date'])
payable_main['year and month'] = payable_main['date'].map(str).str[:7]
payable_main['year and quarter'] = payable_main['date'].map(str).str[:4] + '-Q' + payable_main['date'].dt.quarter.map(str)

24 null(s) after mapping


# Get refund data
Note that this data is by date, not by OID

In [21]:
op_refund = pd.read_sql(
'''
SELECT
DATE(date_purchased) AS date_purchased,
op.orders_id,
SUM((products_quantity - products_quantity_free) * products_price) AS op_rev
FROM orders_products op
JOIN orders o ON op.orders_id = o.orders_id
WHERE DATE(date_purchased) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND orders_status NOT IN (8,9,14,15) # fraud - pending, fraud - confirmed, voided, fraud - voided
GROUP BY op.orders_id
''', db)

col_fix(op_refund)

ot_refund = pd.read_sql(
'''
SELECT
ot.orders_id,
value AS ot_rev
FROM orders_total ot
JOIN orders o ON ot.orders_id = o.orders_id
AND DATE(date_purchased) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND orders_status NOT IN (8,9,14,15) # fraud - pending, fraud - confirmed, voided, fraud - voided
WHERE class = 'ot_subtotal'
''', db)

col_fix(ot_refund)

refunds_main = pd.merge(op_refund, ot_refund, on = 'orders id', how = 'outer')

n = np.sum(refunds_main.isnull().any(1))
if n < 10:
    refunds_main.dropna(inplace = True)
else:
    raise ValueError('you have more than 10 nulls')
    
refunds_main['refund'] = refunds_main['ot rev'] - refunds_main['op rev']

refunds_main.drop(refunds_main[refunds_main['refund'] < 0.001].index, inplace = True)
refunds_main.reset_index(drop = True, inplace = True)

refunds_main['date purchased'] = pd.to_datetime(refunds_main['date purchased'])
for col in ['year and month','year and quarter']:
    refunds_main[col] = jb_dates(refunds_main['date purchased'], col)    

# Get inventory data

In [22]:
# get stock counts
stock_main = pd.read_sql(
'''
SELECT
DATE(timestamp) AS date,
sku_id,
AVG(actual_stock) AS actual_stock
FROM sku_stock_snapshots
WHERE DATE(timestamp) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
GROUP BY sku_id, DATE(timestamp)
''', db)

col_fix(stock_main)

# get sku cost
cost_main = pd.read_sql(
'''
SELECT
A.sku_id,
AVG(A.latest_cost) AS latest_cost # forget about "working", etc., just take the avg per sku
FROM

(SELECT
k.sku_id,
IF(pm.latest_cost = 0 OR k.sku_outsourced_assembly = 1, pm.consigned_latest_cost, pm.latest_cost) AS latest_cost
FROM skus k
LEFT JOIN products_manufacturing pm ON k.sku_id = pm.sku_id) A

GROUP BY A.sku_id
''', db)

col_fix(cost_main)

# map it
stock_main['cost per unit'] = stock_main['sku id'].map(dict(zip(cost_main['sku id'], cost_main['latest cost'])))

# check for nulls
n = stock_main[stock_main.isnull().any(1)]
print('{:,.0f} null(s) after mapping\nor {:,.2f}% of total lines'.format(len(n), len(n)/len(stock_main) * 100))
stock_main.dropna(inplace = True)
stock_main.reset_index(drop = True, inplace = True)

# create some columns
stock_main['date'] = pd.to_datetime(stock_main['date'])
stock_main['year and month'] = [str(x)[:7] for x in stock_main['date']]
stock_main['year and quarter'] = stock_main['date'].map(str).str[:4] + '-Q' + stock_main['date'].dt.quarter.map(str)
stock_main['total cost'] = stock_main['actual stock'] * stock_main['cost per unit']

# get data for cost of goods sold
op_main = pd.read_sql(
'''
SELECT
DATE(date_purchased) AS date,
part_id,
SUM(products_quantity - products_quantity_free) AS qty_bought
FROM orders_products op
JOIN orders o ON op.orders_id = o.orders_id
AND DATE(date_purchased) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND orders_status NOT IN (8,9,10,11,12,14,15)
AND payment_method != 'Replacement Order'
GROUP BY DATE(date_purchased), part_id 
''', db)

col_fix(op_main)

26,011 null(s) after mapping
or 0.28% of total lines


In [23]:
print('{:,.0f} lines in stock_main'.format(len(stock_main)))

9,312,360 lines in stock_main


### Map costs

In [24]:
# map costs
op_main['cost per unit'] = op_main['part id'].map(dict(zip(all_cost_by_part['part id'], all_cost_by_part['latest cost'])))
op_main['total cost'] = op_main['qty bought'] * op_main['cost per unit']

# create these columns
op_main['date'] = pd.to_datetime(op_main['date'])
op_main['year and month'] = [str(x)[:7] for x in op_main['date']]
op_main['year and quarter'] = op_main['date'].map(str).str[:4] + '-Q' + op_main['date'].dt.quarter.map(str)

### Get order counts

In [25]:
date_start2 = str(qb_main['date'].min().date())

orders_main = pd.read_sql(
'''
SELECT
DATE(date_purchased) AS date_purchased,
orders_id
FROM orders
WHERE DATE(date_purchased) BETWEEN ' '''+ date_start2 + ''' ' AND ' '''+ date_end +''' '
AND orders_status NOT IN (8,9,11,14,15) # fraud - pending, fraud - confirmed, replaced defective, voided, fraud - void
''', db)

col_fix(orders_main)

for col in ['year and month','year and quarter']:
    orders_main[col] = jb_dates(orders_main['date purchased'], col)

### Create variables that we will use for analysis

In [26]:
d = {'year and month':[30,13,'m2m',12],
     'year and quarter':[90,9,'q2q',4]}

ratios = {}

for k,v in d.items():
    
    days = v[0]
    
    #==================================================
    # ORDER COUNT
    #==================================================    
    
    order_count = orders_main.groupby(k)[['orders id']].nunique()
    order_count.columns = ['order count']
    
    ratios[k] = {'order count':order_count}
    
    #==================================================
    # TOTAL REVENUE 
    # = all revenue generated
    #==================================================
    
    total_revenue_main = qb_main[(qb_main['income/expense'] == 'income')
                               & (qb_main['amount'] > 0)] # exclude refunds
    
    total_revenue = total_revenue_main.groupby(k)[['amount']].sum()
    total_revenue.columns = ['total revenue']
    
    ratios[k]['total revenue'] = total_revenue
    
    #==================================================
    # AOV
    #==================================================    
    
    aov = pd.DataFrame(total_revenue['total revenue']/order_count['order count'])
    aov.columns = ['aov']
    
    ratios[k]['aov'] = aov
        
    #==================================================
    # RETURNS, etc.
    #==================================================
    
    returns_etc_main = qb_main[(qb_main['income/expense'] == 'income')
                              & (qb_main['amount'] < 0)]
    
    returns_etc = returns_etc_main.groupby(k)[['amount']].sum()
    returns_etc.columns = ['returns, allowances, discounts']
    returns_etc = np.abs(returns_etc)
    
    ratios[k]['returns etc'] = returns_etc
    
    #==================================================
    # NET SALES 
    # = total revenue - ( returns + allowances + discounts )    
    #==================================================
    
    net_sales = pd.DataFrame(total_revenue['total revenue'] - returns_etc['returns, allowances, discounts'])
    net_sales.columns = ['net sales'] 
    
    ratios[k]['net sales'] = net_sales
    
    #==================================================
    # SALES REVENUE 
    # = OPERATNG REVENUE
    # = revenue from selling goods or services
    #==================================================    
    
    sales_revenue_main = qb_main[(qb_main['income/expense'] == 'income')
                          & (qb_main['main category'] == 'kit sales')
                          & (qb_main['amount'] > 0)]
    sales_revenue = sales_revenue_main.groupby(k)[['amount']].sum()
    sales_revenue.columns = ['sales revenue']
    
    ratios[k]['sales revenue'] = sales_revenue
    
    #==================================================
    # COGS
    # = Cost of Sales
    #==================================================
    
    cogs_main = qb_main[qb_main['super category'] == 'cost of goods']
    cogs = cogs_main.groupby(k)[['amount']].sum()
    cogs.columns = ['cost of goods']
    
    ratios[k]['cost of goods'] = cogs
    
    #==================================================
    # GROSS INCOME
    # = Sales Revenue - COGS
    #==================================================
    
    gross_income = pd.DataFrame(sales_revenue['sales revenue'] - cogs['cost of goods'])
    gross_income.columns = ['gross income']
    
    ratios[k]['gross income'] = gross_income
    
    #==================================================
    # GROSS PROFIT
    # = Net Sales - COGS
    #==================================================
    
    gross_profit = pd.DataFrame(net_sales['net sales'] - cogs['cost of goods'])
    gross_profit.columns = ['gross profit']
    
    ratios[k]['gross profit'] = gross_profit
    
    #==================================================
    # GROSS MARGIN
    # = GROSS PROFIT MARGIN
    # = (Net Sales - COGS) / Net Sales
    # = Gross Profit / Net Sales
    #==================================================
    
    gross_profit_margin = pd.DataFrame(gross_profit['gross profit']/net_sales['net sales'])
    gross_profit_margin.columns = ['gross profit margin']
    
    ratios[k]['gross profit margin'] = gross_profit_margin
    
    #==================================================
    # OPERATING EXPENSES
    # = NON MANUFACTURING EXPENSES
    # = expenses incurred from running the biz that are not directly associated with production
    # = payroll, employee benefits and 401K contributions, transportation, travel, rent, etc.
    #==================================================
    
    operating_expenses_main = qb_main[(qb_main['income/expense'] == 'expense') 
                                    & (qb_main['super category'] != 'cost of goods')]
    operating_expenses = operating_expenses_main.groupby([k,'super category'])[['amount']].sum().unstack(1).fillna(0)
    operating_expenses.columns = operating_expenses.columns.droplevel(0)
    operating_expenses['operating expenses'] = operating_expenses.sum(1)
    
    ratios[k]['operating expenses'] = operating_expenses
    
    #==================================================
    # OPERATING INCOME 
    # = EBIT
    # = NET OPERATING INCOME
    # = OPERATING PROFIT
    # = NET OPERATING PROFIT
    # = sales revenue - (COGS + Operating Expenses)
    #==================================================
    
    operating_income = pd.DataFrame(sales_revenue['sales revenue'] - (cogs['cost of goods'] + operating_expenses['operating expenses']))
    operating_income.columns = ['operating income']
    
    ratios[k]['operating income'] = operating_income
    
    #==================================================
    # OPERATING MARGIN 
    # = operating income / net sales
    #==================================================
    
    operating_margin = pd.DataFrame(operating_income['operating income']/net_sales['net sales'])
    operating_margin.columns = ['operating margin']
    
    ratios[k]['operating margin'] = operating_margin
    
    #==================================================
    # NET PROFIT
    # = BOTTOM LINE
    # = PROFIT AFTER TAX
    # = NET INCOME
    # = Net Sales - Total Expenditure
    #==================================================
    
    net_profit = pd.DataFrame(net_sales['net sales'] - (cogs['cost of goods'] + operating_expenses['operating expenses']))
    net_profit.columns = ['net profit']
    net_profit['fiscal year total'] = net_profit.groupby(net_profit.index.str[:4])[['net profit']].cumsum()
    
    ratios[k]['net profit'] = net_profit
    
    #==================================================
    # NET PROFIT MARGIN
    # = net profit / net sales
    #==================================================  
    
    net_profit_margin = net_profit[['net profit']].div(net_sales.values, axis = 0)
    net_profit_margin.columns = ['net profit margin']
    
    ratios[k]['net profit margin'] = net_profit_margin
        
    #==================================================
    # ASSETS
    #==================================================
    
    cols = ['super category',
            'main category',
            'subcategory 01',
            'subcategory 02',
            'subcategory 03',
            'subcategory 04']

    assets_main = balance_main[(balance_main['super category'] == 'assets')
                             & (balance_main['amount'].isnull() == False)]

    ls = []
    
    dates = sorted(list(set(balance_main[k])))
    for i in dates:
        if 'NaT' in i:
            dates.remove(i)    
            
    for d in dates:
        df = assets_main[assets_main[k] <= d].sort_values(cols + [report_type])
        df.drop_duplicates(cols, keep = 'last', inplace = True)
        subtotals = df.groupby(['main category', report_type])[['balance']].sum().unstack(1).sum(1)
        grand_total = subtotals.sum()
        ls.append([d] + subtotals.values.flatten().tolist() + [grand_total])

    cols2 = [k,
            'current assets',
            'fixed assets',
            'total assets']

    assets = pd.DataFrame(ls, columns = cols2)
    assets.sort_values(k, inplace = True)
    assets.set_index(k, inplace = True)    
    
    ix = assets.index.tolist()
    if any('.' in x for x in ix):
        new_ix = [x[:-2] for x in assets.index.tolist()]
        assets.index = new_ix    
    
    ratios[k]['assets'] = assets
    
    #==================================================
    # RETURN ON ASSETS
    # = ROA
    # = see DuPont Ratio for a more in-depth look
    # = net income / total assets
    #==================================================    
    
    roa = pd.DataFrame(net_profit['net profit']/assets['total assets']).dropna()
    roa.columns = ['return on assets']
        
    ratios[k]['roa'] = roa
        
    #==================================================
    # LIABILITIES (no financial debt, so no fixed liabilities)
    #==================================================
    
    cols = ['super category',
            'main category',
            'subcategory 01',
            'subcategory 02',
            'subcategory 03',
            'subcategory 04']

    liabilities_main = balance_main[balance_main['main category'] == 'liabilities'].copy()
    
    ls = []    
    
    dates = sorted(list(set(balance_main[k])))
    for i in dates:
        if 'NaT' in i:
            dates.remove(i)    
            
    for d in dates:
        df = liabilities_main[liabilities_main[k] <= d].sort_values(cols + [report_type])
        df.drop_duplicates(cols, keep = 'last', inplace = True)
        subtotals = df.groupby(['main category', report_type])[['balance']].sum().sum()
        ls.append([d] + subtotals.values.flatten().tolist())

        cols2 = [k, 'liabilities']
        
    liabilities = pd.DataFrame(ls, columns = cols2)
    liabilities.sort_values(k, inplace = True)
    liabilities.set_index(k, inplace = True)
    
    ix = liabilities.index.tolist()
    if any('.' in x for x in ix):
        new_ix = [x[:-2] for x in liabilities.index.tolist()]
        liabilities.index = new_ix        
    
    ratios[k]['liabilities'] = liabilities
    
    #==================================================
    # DEBT RATIO
    # = total liabilities / total assets
    #==================================================
    
    debt_ratio = pd.DataFrame(liabilities['liabilities']/assets['total assets'])
    debt_ratio.columns = ['debt ratio']
    
    ratios[k]['debt ratio'] = debt_ratio
    
    #==================================================
    # CURRENT RATIO
    # = current assets / current liabilities
    #==================================================    
    
    current_ratio = pd.DataFrame(assets['current assets']/liabilities['liabilities'])
    current_ratio.columns = ['current ratio']
    
    ratios[k]['current ratio'] = current_ratio
    
    #==================================================
    # EQUITY
    #==================================================
    
    cols = ['super category',
            'main category',
            'subcategory 01',
            'subcategory 02',
            'subcategory 03',
            'subcategory 04']

    equity_main = balance_main[balance_main['main category'] == 'equity'].copy()
    
    ls = []    
    
    dates = sorted(list(set(balance_main[k])))
    for i in dates:
        if 'NaT' in i:
            dates.remove(i)    
            
    for d in dates:
        df = equity_main[equity_main[k] <= d].sort_values(cols + [report_type])
        df.drop_duplicates(cols, keep = 'last', inplace = True)
        subtotals = df.groupby(['main category', report_type])[['balance']].sum().sum()
        ls.append([d] + subtotals.values.flatten().tolist())

        cols2 = [k, 'equity']
        
    equity = pd.DataFrame(ls, columns = cols2)
    equity.sort_values(k, inplace = True)
    equity.set_index(k, inplace = True)
    
    ix = equity.index.tolist()
    if any('.' in x for x in ix):
        new_ix = [x[:-2] for x in equity.index.tolist()]
        equity.index = new_ix        
        
    # factor in net profit fiscal year total
    ix = equity.index.tolist()
    equity = equity + net_profit.loc[ix][['fiscal year total']].values        
    
    ratios[k]['equity'] = equity
    
    #==================================================
    # WORKING CAPITAL
    # = current assets - current liabilities (we don't have any fixed liabilities)
    #==================================================
    
    working_capital = pd.DataFrame(assets['current assets'] - liabilities['liabilities'])
    working_capital.columns = ['working capital']    
    
    ratios[k]['working capital'] = working_capital
    
    #==================================================
    # RETURN ON CAPITAL EMPLOYED
    # = ROCE
    # = net operating profit / ( total assets - current liabilities )
    # = net operating profit / ( working capital + current assets - current liabilities )
    #==================================================
    
    roce = pd.DataFrame(operating_income['operating income']/(assets['total assets'] - liabilities['liabilities'])).dropna()
    roce.columns = ['return on capital employed']    
    
    ratios[k]['roce'] = roce
        
    #==================================================
    # RECEIVABLES TURNOVER
    #==================================================
    #===== GROSS CREDIT CARD SALES
    
    gross_credit = osh[osh['billing status name'] == 'billed'].groupby(k)[['net amount']].sum()
    gross_credit.columns = ['gross credit sales']
    
    ratios[k]['gross credit'] = gross_credit
    
    #===== PAYMENT RECEIVED
    g1 = osh[osh['billing status name'] == 'payment received'].groupby([k,'date'])[['net amount']].sum()

    # get just the first and last days from whatever period (quarter vs month)
    def first_last(df):
        return df.iloc[[0, -1]]
    g2 = g1.groupby(level = 0, group_keys = False).apply(first_last)

    # create this column by which to unsatck
    new_ix = ['first day','last day'] * int((len(g2.index.get_level_values(1))/2))
    g2['day type'] = new_ix

    # make a new index
    g3 = g2.reset_index().set_index([k,'day type']).drop('date',1)

    # unstack and clean
    g4 = g3.unstack(1)
    g4.columns = g4.columns.droplevel(0)

    t = k.split(' ')[-1]
    g4['avg a/r per ' + t] = g4.mean(1)    
    
    #===== REFUNDS
    refunds = refunds_main.groupby(k)[['refund']].sum()
    refunds.columns = ['refunds']

    #===== MERGE INTO ONE
    cred_main = gross_credit.join(g4[['avg a/r per ' + t]]).join(refunds).fillna(0)
    cred_main['net credit sales'] = cred_main['gross credit sales'] - cred_main['refunds']
    cred_main['receivables turnover'] = cred_main['net credit sales']/cred_main['avg a/r per ' + t]
    cred_main['days to turn'] = days/cred_main['receivables turnover']
    
    ratios[k]['receivables main'] = cred_main
    
    #==================================================
    # PAYABLES TURNOVER
    #==================================================  
    
    a = payable_main.groupby(k)[['net order total']].sum()
    a.columns = ['net payable']

    g1 = payable_main.groupby([k,'date'])[['net order total']].sum()

    # get just the first and last days from whatever period (quarter vs month)
    g2 = g1.groupby(level = 0, group_keys = False).apply(first_last)

    # create this column by which to unsatck
    new_ix = ['first day','last day'] * int((len(g2.index.get_level_values(1))/2))
    g2['day type'] = new_ix

    # make a new index
    g3 = g2.reset_index().set_index([k,'day type']).drop('date',1)

    # unstack and clean
    g4 = g3.unstack(1)
    g4.columns = g4.columns.droplevel(0)

    t = k.split(' ')[-1]
    g4['avg a/p per ' + t] = g4.mean(1)    

    # make into one df
    pay_main = a.join(g4[['avg a/p per ' + t]])
    pay_main['payables turnover'] = pay_main['net payable']/pay_main['avg a/p per ' + t]
    pay_main['days to turn'] = days/pay_main['payables turnover']
    
    ratios[k]['payables main'] = pay_main
    
    #==================================================
    # DEPRECIATION
    #==================================================         
    
    ix = []
    for col in balance_main.columns:
        try:
            df = balance_main[balance_main[col].str.contains('depr')]
            if df.empty == False:
                for i in df.index:
                    ix.append(i)
        except:
            pass

    depreciation_main = balance_main.loc[ix]            
        
    cols = ['super category',
            'main category',
            'subcategory 01',
            'subcategory 02',
            'subcategory 03',
            'subcategory 04']
   
    ls = []    
    
    dates = sorted(list(set(balance_main[k])))
    for i in dates:
        if 'NaT' in i:
            dates.remove(i)    
            
    for d in dates:
        df = depreciation_main[depreciation_main[k] <= d].sort_values(cols + [report_type])
        df.drop_duplicates(cols, keep = 'last', inplace = True)
        subtotals = df.groupby(['main category', report_type])[['balance']].sum().sum()
        ls.append([d] + subtotals.values.flatten().tolist())

        cols2 = [k, 'depreciation']
        
    depreciation = pd.DataFrame(ls, columns = cols2)
    depreciation.sort_values(k, inplace = True)
    depreciation.set_index(k, inplace = True)
    
    ix = depreciation.index.tolist()
    if any('.' in x for x in ix):
        new_ix = [x[:-2] for x in depreciation.index.tolist()]
        depreciation.index = new_ix
        
    for_avg = depreciation.copy()
    for_avg['year'] = for_avg.index.str[:4]
    for_avg = for_avg.groupby('year', as_index = False)[['depreciation']].mean()

    ix = depreciation.index
    depreciation_avg = pd.DataFrame(data = ix, columns = [k])
    depreciation_avg['year'] = depreciation_avg[k].str[:4]
    depreciation_avg = pd.merge(depreciation_avg, for_avg, how = 'left', on = 'year')
    depreciation_avg.drop('year', 1, inplace = True)
    depreciation_avg.rename(columns = {'depreciation':'avg depreciation'}, inplace = True)
    depreciation_avg.set_index(k, inplace = True)    
    
    depreciation = depreciation.join(depreciation_avg)
    
    ratios[k]['depreciation'] = depreciation        

    #==================================================
    # OPERATING CASH FLOW
    # = net profit (net income) + depreciation + amortization 
    #==================================================            
    # https://www.myaccountingcourse.com/financial-ratios/operating-cash-flow
    
    operating_cash_flow = net_profit[['net profit']].join(np.abs(depreciation[['avg depreciation']])).fillna(0)
    operating_cash_flow['operating cash flow'] = operating_cash_flow.sum(1)
    
    #==================================================
    # OPERATING CASH FLOW RATIO
    # = operating cash flow / total liabilities
    #==================================================       
    
    operating_cash_flow_ratio = pd.DataFrame(operating_cash_flow['operating cash flow']/liabilities['liabilities']).dropna()
    operating_cash_flow_ratio.columns = ['operating cash flow ratio']
    
    ratios[k]['operating cash flow ratio'] = operating_cash_flow_ratio
    
    #==================================================
    # INVENTORY TURNOVER and INVENTORY CONVERSION PERIOD
    #==================================================             
    
    # get avg inventory per period
    g1 = stock_main.groupby([k,'date'])[['total cost']].sum()

    # get just the first and last days from whatever period (quarter vs month)
    def first_last(df):
        return df.iloc[[0, -1]]
    g2 = g1.groupby(level = 0, group_keys = False).apply(first_last)

    # create this column by which to unsatck
    new_ix = ['first day','last day'] * int((len(g2.index.get_level_values(1))/2))
    g2['day type'] = new_ix

    # make a new index
    g3 = g2.reset_index().set_index([k,'day type']).drop('date',1)

    # unstack and clean
    g4 = g3.unstack(1)
    g4.columns = g4.columns.droplevel(0)

    t = k.split(' ')[-1]
    g4['avg inventory per ' + t] = g4.mean(1)

    # get COGS
    op = op_main.groupby(k)[['total cost']].sum()
    op.columns = ['cost of goods sold']

    inv_main = op.join(g4[['avg inventory per ' + t]])
    inv_main['inventory turnover'] = inv_main['cost of goods sold']/inv_main['avg inventory per ' + t]
    inv_main['days to turn'] = days/inv_main['inventory turnover']
    
    ratios[k]['inventory main'] = inv_main
    
    #==================================================    
    # CASH CONVERSION CYCLE
    # = days inventory outstanding + days sales outstanding - days payable outstanding
    # = DIO + DSO - DPO
    
    # WHERE
    
    # DIO 
    # = avg inventory / cost of goods sold
    # = 1 / inventory turnover * days in period
    
    # DSO 
    # = avg receivables / total credit sales
    # = 1 / receivables turnover * days in period
    
    # DPO
    # = avg accounts payable / cost of goods sold * days in period
    #==================================================           
    
    # https://corporatefinanceinstitute.com/resources/knowledge/accounting/cash-conversion-cycle/
    dio = 1 / inv_main[['inventory turnover']] * days
    dio.columns = ['dio']
    
    dso = 1 / cred_main[['receivables turnover']] * days
    dso.columns = ['dso']

    dpo = pd.DataFrame(pay_main['avg a/p per ' + t]/inv_main['cost of goods sold']) * days
    dpo.columns = ['dpo']

    ccc = dio.join(dso).join(dpo)
    ccc['cash conversion cycle'] = ccc['dio'] + ccc['dso'] - ccc['dpo']
    
    ratios[k]['cash conversion cycle all'] = ccc
    ratios[k]['cash conversion cycle'] = ccc[['cash conversion cycle']]

### m2m, q2q, yoy, and confidence intervals for avg per period

In [27]:
for k,v in ratios.copy().items(): # year_and_quarter, year_and_month
    for k2, v2 in v.copy().items(): # ratios and dataframes
        df = v2
        
        if k == 'year and quarter':
            n = 5
        elif k == 'year and month':
            n = 13
        
        p2p = df.pct_change()
        
        ratios[k][k2 + ' p2p'] = p2p
            
        yoy = df.pct_change(periods = n-1)
        ratios[k][k2 + ' yoy'] = yoy
        
        conf = jb_conf(df.iloc[-n:]).loc[:'upper'] # take confidence interval over most recent year
        ratios[k][k2 + ' conf'] = conf

### All keys

In [28]:
print_this = 'no'

if print_this == 'yes':
    print(sorted(list(ratios[report_type].keys())))

*****
# PDF report
*****

In [29]:
chart_width = 2
chart_height = 0.5

### GridSpec Example

In [30]:
from matplotlib import gridspec

df = pd.DataFrame({'a':[1,2,3], 'b':[4,5,6]})

f = plt.figure(figsize = (20,5))
gs = gridspec.GridSpec(1, 2 , width_ratios = [1,3])

#=========================
# Table
#=========================

ax0 = plt.subplot(gs[0])
ax0.axis('off')
ax0.axis('tight')
t0 = ax0.table(cellText = df.values, 
          colLabels = ['one','two'], 
          cellLoc = 'center',
          loc = 'center', 
          bbox = [-0.5, 0, 1.5, 1])

t0.set_fontsize(40)
for key, cell in t0.get_celld().items():
    cell.set_linewidth(3)

#=========================
# Chart
#=========================    
    
ax1 = plt.subplot(gs[1])
df.plot(ax = ax1,
        marker = 'x',
        mew = 10,
        lw = 5,
        color = 'blue')

plt.suptitle('Title here', fontsize = 30, y = 1.02, x = 0.10)
plt.savefig(image_path + 'test image', bbox_inches = 'tight')
plt.close()

### Smooth line example

In [31]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

x=np.array([0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2])
y=np.array([0.57,0.85,0.66,0.84,0.59,0.55,0.61,0.76,0.54,0.55,0.48])

x_new = np.linspace(x.min(), x.max(),500)

f = interp1d(x, y, kind='quadratic')
y_smooth=f(x_new)

plt.plot (x_new,y_smooth)
plt.scatter (x, y)    
plt.close()

# Title page

In [32]:
title = report_type.split(' ')[-1].title() + 'ly Performance Report'

ls = sorted(list(set(balance_main.sort_values('date')[report_type])))
for x in ls:
    if 'NaT' in x:
        ls.remove(x)
        
if any('.' in x for x in ls):
    ls = [x[:-2] for x in ls]
    
cur_period = ls[-1]    
cur_period_pretty = jb_dates(pd.Series(cur_period), report_type + ' pretty')[0]

# Summary
* Order Count
* Total Revenue
* AOV
* Gross Profit
* Gross Profit Margin

In [33]:
summary_colors = ['#b2e4f8',
                  '#99dbf6',
                  '#7fd2f4',
                  '#66c9f1',
                  '#4cc0ef',
                  '#32b7ed',
                  '#19aeeb',
                  '#00a6e9']

summary_ls = ['order count',
              'total revenue',
              'aov',
              'gross profit',
              'gross profit margin']

### Create table

In [34]:
ls = []
for x in summary_ls:
    
    if report_type == 'year and month':
        p2p_title = 'm2m'
    else:
        p2p_title = 'q2q'
    
    cur = ratios[report_type][x].iloc[-1,0].item()
    p2p = ratios[report_type][x + ' p2p'].iloc[-1,0].item()
    yoy = ratios[report_type][x + ' yoy'].iloc[-1,0].item()
    lower, mean, upper = ratios[report_type][x + ' conf'].iloc[:,0].values.flatten().tolist()
    
    # confidence intervals taken over most recent 12 months
    if cur >= lower and cur <= upper:
        comment = 'Within typical range'
    else:
        if cur < lower:
            comment = 'Less than typical range'
        else:
            comment = 'Greater than typical range'
            
    ls.append([x, cur, p2p, yoy, mean, comment])
    
cols = ['category',
        cur_period_pretty,
        p2p_title,
        'yoy',
        'one year avg',
        'comment on current value']

summary_results = pd.DataFrame(ls, columns = cols)

ls1 = ['Avg revenue per order',
       'Profit from making and selling goods',
       'Gross profit as a % of revenue',
       'Count of orders',
       'Revenue excluding returns/other deductions']

descriptions = {z[0]:z[1] for z in zip(sorted(summary_ls), ls1)}

summary_results['description'] = summary_results['category'].map(descriptions)

### Create charts

In [35]:
i = 0
for s in summary_ls:
    
    c = summary_colors[i]
    i += 1
    
    if report_type == 'year and month':
        n = 13
    elif report_type == 'year and quarter':
        n = 9

    data = ratios[report_type][s].iloc[-n:,0]
    x = range(len(data))
    y = data.values.flatten().tolist()
    
    x_smooth = np.linspace(np.min(x), np.max(x), 500)
    f = interp1d(x, y, kind = 'quadratic')
    y_smooth = f(x_smooth)

    fig, ax = plt.subplots(1, 1, figsize = (chart_width, chart_height))

    ax.plot(x_smooth,
            y_smooth,
            lw = 3,
            color = c)
    
    ax.scatter(x, y, color = c)
    
    if min(y) < 0:
        y2 = sorted(y)[-6]
        ax.axhline(0, lw = 1, ls = '--', color = 'black')
        ax.annotate('zero',
                    xy = (0.50,y2),
                   fontsize = 10)
        
    ax.fill_between(x_smooth, y_smooth, len(x_smooth)*[ax.get_ylim()[0]], alpha = 0.1, color = c)
    
    xticks = range(len(x))
    ax.set_xlim(np.min(xticks), np.max(xticks))
    ax.set_xticks(xticks)
    ax.set_xticklabels([])
    ax.grid(alpha = 0.50)    
    
    ax.set_yticks([])
    
    ax.set_facecolor('white')
    
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')    
    
    plt.savefig(image_path + 'Summary - ' + s, bbox_inches = 'tight')
    plt.close()

### Make pretty for HTML

In [36]:
summary_results_table = summary_results.copy()

#=========================
# DEFINE FORMATS
#=========================

ls1 = sorted(summary_ls)
ls2 = ['${:,.2f}','${:,.0f}','{:,.1f}%','{:,.0f}','${:,.0f}']
ls3 = [1,1,100,1,1]
fmt = {z[0]:list(z[1:]) for z in zip(ls1,ls2,ls3)}   

#=========================
# FORMAT VALUES
#=========================

for cat in summary_ls:
    
    for n in [1,4]:
        
        val = summary_results_table[summary_results_table['category'] == cat].iloc[:,n].item()
        f = fmt[cat][0]
        mult = fmt[cat][1]

        if val < 0:
            val2 = '-' + f.format(np.abs(val) * mult)
        else:
            val2 = f.format(val * mult)
        
        col = summary_results_table.columns[n]
        
        summary_results_table.loc[summary_results_table['category'] == cat, summary_results_table.columns == col] = val2
           
#=========================
# FORMAT P2P AND YOY
#=========================    
    
for n in [2,3]:
    vals = summary_results_table.iloc[:,n].tolist()
    vals2 = []
    for v in vals:
        vals2.append('{:,.1f}%'.format(v * 100))
    summary_results_table.iloc[:,n] = vals2    
    

#=========================
# ADD CHARTS
#=========================            

img_ls = []

for cat in summary_results_table['category']:
    img = '<img src = "file://' + image_path + 'Summary - ' + cat + '.png">'
    img_ls.append(img)

summary_results_table['previous two years'] = img_ls

#=========================
# FORMAT CATEGORY NAMES
#=========================        
        
summary_results_table['category'] = summary_results_table['category'].str.title()

d = {'Roa':'ROA',
    'Roce':'ROCE'}

for k,v in d.items():
    summary_results_table['category'] = summary_results_table['category'].str.replace(k,v)            

#=========================
# FORMAT COLUMN NAMES
#=========================        

new_cols = []
for x in summary_results_table.columns:
    if x == 'q2q':
        x = 'Q2Q'
    elif x == 'm2m':
        x = 'M2M'
    elif x == 'yoy':
        x = 'YoY'
    else:
        x = x.title()
        
    new_cols.append(x)
    
summary_results_table.columns = new_cols

In [37]:
summary_results_table

,Category,Feb 2019,M2M,YoY,One Year Avg,Comment On Current Value,Description,Previous Two Years
0,Order Count,"23,447",-7.4%,-6.5%,"24,723",Within typical range,Count of orders,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
1,Total Revenue,"$3,345,193",-17.1%,-6.3%,"$3,854,867",Less than typical range,Revenue excluding returns/other deductions,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
2,Aov,$142.67,-10.5%,0.3%,$157.19,Less than typical range,Avg revenue per order,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
3,Gross Profit,"$1,932,971",-40.6%,14.8%,"$2,224,839",Within typical range,Profit from making and selling goods,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
4,Gross Profit Margin,58.0%,-36.2%,22.3%,58.8%,Within typical range,Gross profit as a % of revenue,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."


### Send to HTML

In [38]:
with pd.option_context('display.max_colwidth', -1):
    summary_results_table = summary_results_table.to_html(index = False, escape = False)

# Profitability ratios
* gross profit
* gross profit margin
* net profit
* net profit margin
* operating margin
* roa
* roce

In [39]:
profit_colors = ['#00ffff',
                 '#00e5e5',
                 '#00cccc',
                 '#00b2b2',
                 '#009999',
                 '#007f7f',
                 '#006666',
                 '#004c4c',
                 '#003333',
                 '#001919']

profit_ls = ['gross profit',
             'gross profit margin',
             'net profit',
             'net profit margin',
             'operating margin',
             'roa',
             'roce']

### Create table

In [40]:
ls = []
for x in profit_ls:
    
    if report_type == 'year and month':
        p2p_title = 'm2m'
    else:
        p2p_title = 'q2q'
    
    cur = ratios[report_type][x].iloc[-1,0].item()
    p2p = ratios[report_type][x + ' p2p'].iloc[-1,0].item()
    yoy = ratios[report_type][x + ' yoy'].iloc[-1,0].item()
    lower, mean, upper = ratios[report_type][x + ' conf'].iloc[:,0].values.flatten().tolist()
    
    # confidence intervals taken over most recent 12 months
    if cur >= lower and cur <= upper:
        comment = 'Within typical range'
    else:
        if cur < lower:
            comment = 'Less than typical range'
        else:
            comment = 'Greater than typical range'
            
    ls.append([x, cur, p2p, yoy, mean, comment])
    
cols = ['category',
        cur_period_pretty,
        p2p_title,
        'yoy',
        'one year avg',
        'comment on current value']

profit_results = pd.DataFrame(ls, columns = cols)

ls1 = ['Profit from making and selling goods',
       'Gross profit as a % of revenue',
       'Overall profit',
       'Overall profit as a % of revenue',
       'Profit from operations',
       'Profit per asset (cash, machines)',
       'Profit per capital employed']

descriptions = {z[0]:z[1] for z in zip(sorted(profit_ls), ls1)}

profit_results['description'] = profit_results['category'].map(descriptions)

### Create charts

In [41]:
i = 0
for p in profit_ls:
    
    c = profit_colors[i]
    i += 1
    
    if report_type == 'year and month':
        n = 13
    elif report_type == 'year and quarter':
        n = 9

    data = ratios[report_type][p].iloc[-n:,0]
    x = range(len(data))
    y = data.values.flatten().tolist()
    
    x_smooth = np.linspace(np.min(x), np.max(x), 500)
    f = interp1d(x, y, kind = 'quadratic')
    y_smooth = f(x_smooth)

    fig, ax = plt.subplots(1, 1, figsize = (chart_width, chart_height))

    ax.plot(x_smooth,
            y_smooth,
            lw = 3,
            color = c)
    
    ax.scatter(x, y, color = c)
    
    if min(y) < 0:
        y2 = sorted(y)[-6]
        ax.axhline(0, lw = 1, ls = '--', color = 'black')
        ax.annotate('zero',
                    xy = (0.50,y2),
                   fontsize = 10)
        
    ax.fill_between(x_smooth, y_smooth, len(x_smooth)*[ax.get_ylim()[0]], alpha = 0.1, color = c)
    
    xticks = range(len(x))
    ax.set_xlim(np.min(xticks), np.max(xticks))
    ax.set_xticks(xticks)
    ax.set_xticklabels([])
    ax.grid(alpha = 0.50)    
    
    ax.set_yticks([])
    
    ax.set_facecolor('white')
    
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')    
    
    plt.savefig(image_path + 'Profitability - ' + p, bbox_inches = 'tight')
    plt.close()

### Make pretty for HTML

In [42]:
profit_results_table = profit_results.copy()

#=========================
# DEFINE FORMATS
#=========================

ls1 = sorted(profit_ls)
ls2 = ['${:,.0f}','{:,.1f}%','${:,.0f}','{:,.1f}%','{:,.1f}%','{:,.1f}%','{:,.1f}%']
ls3 = [1,100,1,100,100,100,100]
fmt = {z[0]:list(z[1:]) for z in zip(ls1,ls2,ls3)}   

#=========================
# FORMAT VALUES
#=========================

for cat in profit_ls:
    
    for n in [1,4]:
        
        val = profit_results_table[profit_results_table['category'] == cat].iloc[:,n].item()
        f = fmt[cat][0]
        mult = fmt[cat][1]

        if val < 0:
            val2 = '-' + f.format(np.abs(val) * mult)
        else:
            val2 = f.format(val * mult)
        
        col = profit_results_table.columns[n]
        
        profit_results_table.loc[profit_results_table['category'] == cat, profit_results_table.columns == col] = val2

#=========================
# FORMAT P2P AND YOY
#=========================    
    
for n in [2,3]:
    vals = profit_results_table.iloc[:,n].tolist()
    vals2 = []
    for v in vals:
        vals2.append('{:,.1f}%'.format(v * 100))
    profit_results_table.iloc[:,n] = vals2    
    

#=========================
# ADD CHARTS
#=========================            

img_ls = []

for cat in profit_results_table['category']:
    img = '<img src = "file://' + image_path + 'Profitability - ' + cat + '.png">'
    img_ls.append(img)

profit_results_table['previous two years'] = img_ls

#=========================
# FORMAT CATEGORY NAMES
#=========================        
        
profit_results_table['category'] = profit_results_table['category'].str.title()

d = {'Roa':'ROA',
    'Roce':'ROCE'}

for k,v in d.items():
    profit_results_table['category'] = profit_results_table['category'].str.replace(k,v)            

#=========================
# FORMAT COLUMN NAMES
#=========================        

new_cols = []
for x in profit_results_table.columns:
    if x == 'q2q':
        x = 'Q2Q'
    elif x == 'm2m':
        x = 'M2M'
    elif x == 'yoy':
        x = 'YoY'
    else:
        x = x.title()
        
    new_cols.append(x)
    
profit_results_table.columns = new_cols

In [43]:
profit_results_table

,Category,Feb 2019,M2M,YoY,One Year Avg,Comment On Current Value,Description,Previous Two Years
0,Gross Profit,"$1,932,971",-40.6%,14.8%,"$2,224,839",Within typical range,Profit from making and selling goods,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
1,Gross Profit Margin,58.0%,-36.2%,22.3%,58.8%,Within typical range,Gross profit as a % of revenue,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
2,Net Profit,"$424,252",-69.5%,"1,992.0%","$279,702",Within typical range,Overall profit,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
3,Net Profit Margin,12.7%,-67.2%,"2,129.6%",6.8%,Within typical range,Overall profit as a % of revenue,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
4,Operating Margin,12.4%,-75.6%,"-5,607.9%",8.1%,Within typical range,Profit from operations,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
5,ROA,5.2%,-70.2%,"1,619.5%",3.5%,Within typical range,"Profit per asset (cash, machines)","<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."
6,ROCE,5.3%,-78.1%,"-4,153.6%",4.3%,Within typical range,Profit per capital employed,"<img src = ""file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis..."


### Send To HTML

In [44]:
with pd.option_context('display.max_colwidth', -1):
    profit_results_table = profit_results_table.to_html(index = False, escape = False)

# Activity Ratios
* receivables turnover
* receivables conversion period


* payables turnover
* payables conversion period


* inventory turnover
* inventory conversion period

In [45]:
activity_colors = ['#ffe5ff',
                   '#ffccff',
                   '#ffb2ff',
                   '#ff99ff',
                   '#ff7fff',
                   '#ff66ff',
                   '#ff4cff',
                   '#ff32ff',
                   '#ff19ff',
                   '#ff00ff']

activity_ls = ['receivables main',
               'payables main',
               'inventory main']

### Create table and charts

In [46]:
activity_results = pd.DataFrame()
ls = []
j = 1
for a in activity_ls:
    
    #=========================
    # CURRENT VALUES
    #=========================    
    
    name = a.split(' ')[0]
    
    cur = ratios[report_type][a][[name + ' turnover','days to turn']].iloc[-1].values.tolist()
    p2p = ratios[report_type][a + ' p2p'][[name + ' turnover','days to turn']].iloc[-1].values.tolist()
    yoy = ratios[report_type][a + ' yoy'][[name + ' turnover','days to turn']].iloc[-1].values.tolist()
        
    #=========================
    # CREATE DATAFRAME
    #=========================        
    
    if report_type == 'year and month':
        p2p_write = 'm2m'
    elif report_type == 'year and quarter':
        p2p_write = 'q2q'
    
    cols = [cur_period_pretty,
            p2p_write,
           'yoy']
    
    idx = pd.MultiIndex.from_product([[name], 
                                      ['turnover','days to turn']],
                                      names = ['category','subcategory'])
    df = pd.DataFrame([[cur[0],p2p[0],yoy[0]],
                       [cur[1],p2p[1],yoy[1]]],
                      idx,
                      cols)
    
    #=========================
    # AVG and INTERVAL
    #=========================    
    
    conf = ratios[report_type][a + ' conf'][[name + ' turnover','days to turn']]
    subcat_ls = ['turnover','days to turn']
    
    df['one year avg'] = np.nan
    df['comment on current value'] = np.nan
    
    # for both "turnover" and "days to turn", do this
    for i in range(2):
        
        lower, mean, upper = conf.iloc[:,i].values.tolist()
        
        if cur[i] >= lower and cur[i] <= upper:
            comment = 'Wihin typical range'
        else:
            if cur[i] < lower:
                comment = 'Less than typical range'
            else:
                comment = 'Greater than typical range'
                
        df.loc[(name, subcat_ls[i]), 'one year avg'] = mean
        df.loc[(name, subcat_ls[i]), 'comment on current value'] = comment
        
    #=========================
    # JOIN TO CREATE A SINGLE DATAFRAME
    #=========================         
    
    activity_results = activity_results.append(df, ignore_index = False)
        
    #=========================
    # CREATE CHARTS
    #=========================            

    c = activity_colors[j * 3]

    charts = ratios[report_type][a][[name + ' turnover','days to turn']]

    turn = charts[name + ' turnover']
    turn_x = range(len(turn))
    turn_y = turn.values.tolist()
    
    turn_x_smooth = np.linspace(np.min(turn_x), np.max(turn_x), 500)
    f = interp1d(turn_x, turn_y, kind = 'quadratic')
    turn_y_smooth = f(turn_x_smooth)

    fig, ax = plt.subplots(figsize = (chart_width, chart_height))
    ax.plot(turn_x_smooth,
            turn_y_smooth,
            lw = 3,
            color = c)
    
    ax.scatter(turn_x, turn_y, color = c)

    xticks = range(len(turn_x))
    ax.set_xlim(np.min(xticks), np.max(xticks))
    ax.set_xticks(xticks)
    ax.set_xticklabels([])
    
    ax.grid(alpha = 0.50)
    
    ax.set_yticks([])
    
    ax.set_facecolor('white')
    
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')  
    
    ax.fill_between(turn_x_smooth, turn_y_smooth, len(turn_y_smooth)*[ax.get_ylim()[0]], alpha = 0.1, color = c)
    
    if min(turn_y) < 0:
        y2 = sorted([v for v in sorted(turn_y) if v > 0])[0]
        ax.axhline(0, lw = 1, ls = '--', color = 'black')
        ax.annotate('zero',
                    xy = (0.50,y2),
                   fontsize = 10)    
    
    plt.savefig(image_path + 'Activity - ' + a.split(' ')[0] + ' - turnover', bbox_inches = 'tight')    
    plt.close()    

    fig, ax = plt.subplots(figsize = (chart_width, chart_height))
    
    days = charts['days to turn']
    x = range(len(days))
    y = days.values.tolist()
    
    ax.bar(x, y,
            color = c,
            edgecolor = 'black',
            lw = 2,
            alpha = 0.25)
    xticks = range(len(x))
    ax.set_xlim(np.min(xticks) - 0.5, np.max(xticks) + 0.5)
    ax.set_facecolor('white')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')  
    plt.savefig(image_path + 'Activity - ' + a.split(' ')[0] + ' - days to turn', bbox_inches = 'tight')    

    j += 1
    
    plt.close()    

### Make pretty for HTML

In [47]:
activity_results_table = activity_results.copy()

#=========================
# FORMAT VALUES
#=========================            

for col in [cur_period_pretty, 'one year avg']:
    activity_results_table[col] = ['{:,.1f}'.format(x) for x in activity_results_table[col]]
    
for col in [p2p_write, 'yoy']:
    activity_results_table[col] = ['{:,.1f}%'.format(x * 100) for x in activity_results_table[col]]
    
#=========================
# ADD CHARTS
#=========================                

img_ls = []

for cat1 in activity_results_table.index.get_level_values(0).unique():
    for cat2 in activity_results_table.index.get_level_values(1).unique():
        img = '<img src = "file://' + image_path + 'Activity - ' + cat1 + ' - ' + cat2 + '.png">'
        img_ls.append(img)

activity_results_table['previous two years'] = img_ls

#=========================
# FIX COLUMN NAMES
#=========================

cols = []
for c in activity_results_table.columns.tolist():
    if c in ['q2q','m2m']:
        c = c.upper()
    elif c == 'yoy':
        c = 'YoY'
    else:
        c = c.title()
    cols.append(c)
    
activity_results_table.columns = cols    

#=========================
# FIX INDEX NAMES
#=========================

ix = list(activity_results_table.index.names)
ix = [x.title() for x in ix]
activity_results_table.index.names = ix

#=========================
# FIX CATEGORY NAMES
#=========================

l0 = activity_results_table.index.get_level_values(0).tolist()
l0 = [x.title() for x in l0]

l1 = activity_results_table.index.get_level_values(1).tolist()
l1 = [x.title() for x in l1]

activity_results_table.set_index([l0,l1], inplace = True)

In [48]:
activity_results_table

Feb 2019       M2M       YoY One Year Avg  \
Receivables Turnover          5.6    -97.3%    -56.9%         41.0   
            Days To Turn      5.3  3,640.2%    132.3%          1.8   
Payables    Turnover         46.9     48.2%    -95.5%        216.4   
            Days To Turn      0.6    -32.5%  2,119.3%          1.4   
Inventory   Turnover          0.2      4.1%      3.4%          0.2   
            Days To Turn    144.2     -4.0%     -3.3%        142.7   

                            Comment On Current Value  \
Receivables Turnover         Less than typical range   
            Days To Turn  Greater than typical range   
Payables    Turnover             Wihin typical range   
            Days To Turn         Wihin typical range   
Inventory   Turnover             Wihin typical range   
            Days To Turn         Wihin typical range   

                                                                                                           Previous Two Years  
Receivables Turnover      <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...  
            Days To Turn  <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...  
Payables    Turnover      <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...  
            Days To Turn  <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...  
Inventory   Turnover      <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...  
            Days To Turn  <img src = "file:///Users/jarad/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis...

### Send to HTML

In [49]:
with pd.option_context('display.max_colwidth', -1):
    activity_results_table = activity_results_table.to_html(index = True, escape = False)

# Liquidity Ratios
* Debt ratio
* Current ratio/working captial ratio
* Operating case flow ratio
* Cash conversion cycle

In [50]:
liquidity_colors = ['#efeaf4',
                    '#e0d5e9',
                    '#d1c0de',
                    '#c1abd3',
                    '#a381bd',
                    '#936cb2',
                    '#8456a7',
                    '#75429c',
                    '#662d91']

liquidity_ls = ['debt ratio',
               'current ratio',
               'operating cash flow ratio',
               'cash conversion cycle']

### Create table

In [51]:
ls = []
for x in liquidity_ls:
    
    if report_type == 'year and month':
        p2p_title = 'm2m'
    else:
        p2p_title = 'q2q'
    
    cur = ratios[report_type][x].iloc[-1].item()
    p2p = ratios[report_type][x + ' p2p'].iloc[-1].item()
    yoy = ratios[report_type][x + ' yoy'].iloc[-1].item()
    lower, mean, upper = ratios[report_type][x + ' conf'].iloc[:,0].tolist()
    
    # confidence intervals taken over most recent 12 months
    if cur >= lower and cur <= upper:
        comment = 'Within typical range'
    else:
        if cur < lower:
            comment = 'Less than typical range'
        else:
            comment = 'Greater than typical range'
            
    ls.append([x, cur, p2p, yoy, mean, comment])
    
cols = ['category',
        cur_period_pretty,
        p2p_title,
        'yoy',
        'one year avg',
        'comment on current value']

liquidity_results = pd.DataFrame(ls, columns = cols)

ls1 = ['Days to convert inventory/resources into cash from sales',
       'How well assets cover liabilities',
       'How much liabilties outweigh assets',
       'How much cash to cover debt']

descriptions = {z[0]:z[1] for z in zip(sorted(liquidity_ls), ls1)}

liquidity_results['description'] = liquidity_results['category'].map(descriptions)

### Create charts

In [52]:
i = 1
for l in liquidity_ls:
    
    c = liquidity_colors[i]
    i += 2
    
    if report_type == 'year and month':
        n = 13
    elif report_type == 'year and quarter':
        n = 9

    data = ratios[report_type][l].iloc[-n:]
    x = range(len(data))
    y = data.values.flatten().tolist()

    x_smooth = np.linspace(np.min(x), np.max(x), 500)
    f = interp1d(x, y, kind = 'quadratic')
    y_smooth = f(x_smooth)
    
    fig, ax = plt.subplots(1, 1, figsize = (chart_width, chart_height))

    ax.plot(x_smooth,
            y_smooth,
            lw = 3,
            color = c)
    
    ax.scatter(x, y, color = c)
    
    if min(y) < 0:
        y2 = sorted(y)[-6]
        ax.axhline(0, lw = 1, ls = '--', color = 'black')
        ax.annotate('zero',
                    xy = (0.50,y2),
                   fontsize = 10)
        
    ax.fill_between(x_smooth, y_smooth, len(x_smooth)*[ax.get_ylim()[0]], alpha = 0.1, color = c)
    
    xticks = range(len(x))
    ax.set_xlim(np.min(xticks), np.max(xticks))
    ax.set_xticks(xticks)
    ax.set_xticklabels([])
    ax.grid(alpha = 0.50)    
    
    ax.set_yticks([])
    
    ax.set_facecolor('white')
    
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')    
    
    plt.savefig(image_path + 'Liquidity - ' + l, bbox_inches = 'tight')
    plt.close()

### Make pretty for HTML

In [53]:
#=========================
# FORMAT VALUES
#=========================    

liquidity_results_table = liquidity_results.copy()

for col in [cur_period_pretty, 'one year avg']:
    liquidity_results_table[col] = ['{:,.1f}'.format(x) for x in liquidity_results_table[col]]

for col in [p2p_write, 'yoy']:
    liquidity_results_table[col] = ['{:,.1f}%'.format(x * 100) for x in liquidity_results_table[col]]
    
#=========================
# ADD CHARTS
#=========================            

img_ls = []

for cat in liquidity_results_table['category']:
    img = '<img src = "file://' + image_path + 'Liquidity - ' + cat + '.png">'
    img_ls.append(img)

liquidity_results_table['previous two years'] = img_ls

#=========================
# FORMAT CATEGORY NAMES
#=========================        
        
liquidity_results_table['category'] = liquidity_results_table['category'].str.title()

d = {'Roa':'ROA',
    'Roce':'ROCE'}

for k,v in d.items():
    liquidity_results_table['category'] = liquidity_results_table['category'].str.replace(k,v)            

#=========================
# FORMAT COLUMN NAMES
#=========================        

new_cols = []
for x in liquidity_results_table.columns:
    if x == 'q2q':
        x = 'Q2Q'
    elif x == 'm2m':
        x = 'M2M'
    elif x == 'yoy':
        x = 'YoY'
    else:
        x = x.title()
        
    new_cols.append(x)
    
liquidity_results_table.columns = new_cols

### Send to HTML

In [54]:
with pd.option_context('display.max_colwidth', -1):
    liquidity_results_table = liquidity_results_table.to_html(index = False, escape = False)

### Calculations and descriptions

In [55]:
d = {'Debt Ratio':'total assets / liabilities',
    'Current Ratio':'current assets / liabilities',
    'Operating Cash Flow Ratio':'( net profit + depreciation ) / liabilities',
    'Cash Conversion Cycle':'days inventory outstanding + days sales outstanding - days payable outstanding'}

df1 = pd.DataFrame(list(d.items()), columns = ['category','calculation'])

d = {'Debt Ratio': 'How many assets per one liability, where assets include things like cash, equipment, and inventory',
    'Current Ratio': 'How many current assets per one liability, where a current asset includes cash, receivables, and inventory',
    'Operating Cash Flow Ratio':'How much profit + machine worth per dollar of liabilities.',
    'Cash Conversion Cycle':'Measures the duration of time for which each net input dollar (cash) is tied up in the production and sales process before it gets converted into cash received through sales made to customers.'}

df2 = pd.DataFrame(list(d.items()), columns = ['category','description'])

liquidity_details = pd.merge(df1, df2, on = 'category')
liquidity_details.columns = liquidity_details.columns.str.title()

with pd.option_context('display.max_colwidth', -1):
    liquidity_details = liquidity_details.to_html(index = False)

### Hand-written summary

In [57]:
path = tilde + '/Scripts/Fake Folder/Finance/Recurring/The Pillars/Ratio Analysis/Docs/hand-written summary.txt'
hand_written_summary = open(path, 'r').read()

print(hand_written_summary)

Overall, total revenue, AOV, and gross profit were each below their typical range, while gross profit and the gross profit margin were within their typical range, implying an increase in profitability. Indeed, net profit (the bottom line) increased nearly 15% YoY, up from $20,300 to $424,250.
<br><br>
As for turnover, on avg we collected our debt every five days, we paid our debt once per day, and it took 144 days to turn our inventory, which is about 1.5 times longer than the goal of 90 days.
<br><br>
Moving on to liquidity ratios, which show how effectively assets cover liabilities, our debt ratio was near zero, which means we had much more in assets than in liabilities. This was supported by our high current ratio of ~24, which shows us that for each dollar in liabilities we had $24 in assets. Finally, to turn our inventory and resources into cash it took 150 days, which has been about the same for the last year.


# PDF Start

In [65]:
if pdf_write == 'yes':

    from jinja2 import Environment, FileSystemLoader
    env = Environment(loader = FileSystemLoader(html_path))

    template = env.get_template(html_title)

    template_vars = {'title':title,
                    'subtitle':cur_period_pretty,
                    'now_pretty':now_pretty,
                    'hand_written_summary':hand_written_summary,
                     
                    'summary_results_table':summary_results_table,
                    
                    'profit_results_table':profit_results_table,            
                     
                    'activity_results_table':activity_results_table,
                    
                    'liquidity_results_table':liquidity_results_table}

    html_out = template.render(template_vars)
    
    from weasyprint import HTML
    HTML(string = html_out).write_pdf(pdf_path + report_title + '.pdf',
                                      stylesheets = [style_sheet_path])                               

In [66]:
print('done')

done
